In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time

from google.colab import files


uploaded = files.upload()


Saving archive.zip to archive.zip
CPU times: user 8.38 s, sys: 911 ms, total: 9.29 s
Wall time: 9min 25s


In [ ]:
uploaded

In [ ]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "/content/archive.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
IMDB Dataset.csv                               2019-10-19 19:24:58     66212309
Extracting all the files now...
Done!


In [ ]:
df=pd.read_csv('IMDB Dataset.csv',nrows=1000)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(1000, 2)

In [ ]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
batch_1=df

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
batch_1['review_tknz'] = batch_1['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,
                                                                             max_length=512,
                                                                             truncation=True,
                                                                             padding="max_length")))

CPU times: user 10.5 s, sys: 64.7 ms, total: 10.5 s
Wall time: 15 s


In [ ]:
batch_1

,review,sentiment,review_tknz
0,One of the other reviewers has mentioned that ...,positive,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385..."
1,A wonderful little production. <br /><br />The...,positive,"[101, 1037, 6919, 2210, 2537, 1012, 1026, 7987..."
2,I thought this was a wonderful way to spend ti...,positive,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126..."
3,Basically there's a family where a little boy ...,negative,"[101, 10468, 2045, 1005, 1055, 1037, 2155, 207..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[101, 9004, 3334, 4717, 7416, 1005, 1055, 1000..."
...,...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive,"[101, 2498, 2003, 6730, 1012, 2074, 3198, 1463..."
996,I hated it. I hate self-aware pretentious inan...,negative,"[101, 1045, 6283, 2009, 1012, 1045, 5223, 2969..."
997,I usually try to be professional and construct...,negative,"[101, 1045, 2788, 3046, 2000, 2022, 2658, 1998..."
998,If you like me is going to see this in a film ...,negative,"[101, 2065, 2017, 2066, 2033, 2003, 2183, 2000..."


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb

In [ ]:
padded=list(batch_1['review_tknz'])

In [ ]:
padded=np.array(padded)

In [ ]:
padded.shape

(1000, 512)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 512)

In [ ]:
padded.shape

(1000, 512)

In [ ]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])

In [ ]:
padded

array([[ 101, 2028, 1997, ...,    0,    0,    0],
       [ 101, 1037, 6919, ...,    0,    0,    0],
       [ 101, 1045, 2245, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2788, ...,    0,    0,    0],
       [ 101, 2065, 2017, ...,    0,    0,    0],
       [ 101, 2023, 2003, ..., 1006, 2036,  102]])

In [ ]:
%%time
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 6.26 ms


In [ ]:
###Splitting the 1000 records(rows) into 10 small arrays each containing 100 records to avoid memory crash

In [ ]:
input_ids_list=torch.split(input_ids,100)
attention_mask_list = torch.split(attention_mask,100)

In [ ]:
input_ids_list=list(input_ids_list)
attention_mask_list=list(attention_mask_list)

In [ ]:
hidden_states=[]

In [ ]:
input_ids_list[9].shape

torch.Size([100, 512])

In [ ]:
len(input_ids_list)

10

In [ ]:
#running through the model batchwise

In [ ]:
%%time
for i in range(len(input_ids_list)):
  with torch.no_grad():
    last_hidden_states = model(input_ids_list[i], attention_mask=attention_mask_list[i])
    hidden_states.append(last_hidden_states)

CPU times: user 29min 48s, sys: 22.9 s, total: 30min 11s
Wall time: 30min 20s


In [ ]:
hidden_states

In [ ]:
##stacking the outputs into one matrix 

In [ ]:
final=torch.stack(hidden_states, 0)

In [ ]:
final.shape

In [ ]:
# %%time
# input_ids = torch.tensor(padded)  
# attention_mask = torch.tensor(attention_mask)

# with torch.no_grad():
#     last_hidden_states = model(input_ids, attention_mask=attention_mask)

CPU times: user 2min 34s, sys: 1.79 s, total: 2min 36s
Wall time: 2min 41s


In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1['sentiment']

In [ ]:
###splitting the data into train and test

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.84

In [ ]:
test_predicted=lr_clf.predict(test_features)

In [ ]:
print(classification_report(test_labels,test_predicted))

              precision    recall  f1-score   support

    negative       1.00      0.73      0.85        15
    positive       0.71      1.00      0.83        10

    accuracy                           0.84        25
   macro avg       0.86      0.87      0.84        25
weighted avg       0.89      0.84      0.84        25

